#Titanic Dataset Exercise

## Import libraries and Data
- Import the libraries you'll need to perform the exercises
- Import the data, producing spark dataframes. NB the options used e.g. header. Make sure to display your dataframes to check they are as expected

In [3]:
""" Import Libraries """
import pyspark.sql.functions as f
import pyspark.sql.types as t
from pyspark.sql.window import Window

""" Import the data """
# File location and type
file_location = "/FileStore/tables/"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_train = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location + 'train.csv')

df_test = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location + 'test.csv')

df_gender_schema = t.StructType([
                    t.StructField("PassengerId", t.IntegerType()),
                    t.StructField("Survived", t.IntegerType())
                ])

df_gender = spark.read.format(file_type) \
  .option("inferSchema", False) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .schema(df_gender_schema) \
  .load(file_location + 'gender_submission.csv')



In [4]:
df_train.show(n=5, truncate = False)

+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
PassengerId|Survived|Pclass|Name |Sex |Age |SibSp|Parch|Ticket |Fare |Cabin|Embarked|
+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
1 |0 |3 |Braund, Mr. Owen Harris |male |22.0|1 |0 |A/5 21171 |7.25 |null |S |
2 |1 |1 |Cumings, Mrs. John Bradley (Florence Briggs Thayer)|female|38.0|1 |0 |PC 17599 |71.2833|C85 |C |
3 |1 |3 |Heikkinen, Miss. Laina |female|26.0|0 |0 |STON/O2. 3101282|7.925 |null |S |
4 |1 |1 |Futrelle, Mrs. Jacques Heath (Lily May Peel) |female|35.0|1 |0 |113803 |53.1 |C123 |S |
5 |0 |3 |Allen, Mr. William Henry |male |35.0|0 |0 |373450 |8.05 |null |S |
+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 5 rows

##Questions
### 1) Excluding the child passengers (Age < 18) find the the minimum, maximum, and average fares for each passenger class.

1) Create a complete dataset by: <br>
<Tab> a. Join `df_gender` with `df_test` on `PassengeId` <br>
<Tab> b. Append the dataframe from a with `df_train`
  
Tip: Check that the transformations you are doing produce the expected output e.g. by checking number of rows, columns, schema.
  
2) Filter the complete dataframe `df` selecting only passenger with `Age < 18`, aggregate by `Pclass` calculating the minimum, maximum and average fares.

In [7]:
# Join df_gender to df_test
df_join = df_test.join(df_gender, df_test.PassengerId == df_gender.PassengerId, 'left_outer').drop(df_gender.PassengerId)

#Check we have the same number of rows in df_join as df_test since we are using a left outer join.
assert df_join.count() == df_gender.count()

# Union the df_train with df_join
df = df_train.unionAll(df_join.select(*df_train.columns))

#Check we get the correct number of rows and the schema is correct
assert df.count() == df_train.count() + df_join.count()
assert df.schema == df_train.schema
df.show(n = 5, truncate = False)

+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
PassengerId|Survived|Pclass|Name |Sex |Age |SibSp|Parch|Ticket |Fare |Cabin|Embarked|
+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
1 |0 |3 |Braund, Mr. Owen Harris |male |22.0|1 |0 |A/5 21171 |7.25 |null |S |
2 |1 |1 |Cumings, Mrs. John Bradley (Florence Briggs Thayer)|female|38.0|1 |0 |PC 17599 |71.2833|C85 |C |
3 |1 |3 |Heikkinen, Miss. Laina |female|26.0|0 |0 |STON/O2. 3101282|7.925 |null |S |
4 |1 |1 |Futrelle, Mrs. Jacques Heath (Lily May Peel) |female|35.0|1 |0 |113803 |53.1 |C123 |S |
5 |0 |3 |Allen, Mr. William Henry |male |35.0|0 |0 |373450 |8.05 |null |S |
+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 5 rows

In [8]:
# Filter for over 18s, Use groupBy and agg functions to aggregate over class using pyspark sql functions min, max, mean
df_agg_fare = df.where(df.Age>=18).groupBy(['Pclass']).agg(f.round(f.min('Fare'),2).alias('min_fare'),
                                                 f.round(f.max('Fare'),2).alias('max_fare'),
                                                 f.round(f.mean('Fare'),2).alias('mean_fare'))
df_agg_fare.show()

#Using sql
df.createOrReplaceTempView('table_df')
sql_query = 'SELECT Pclass, round(min(Fare),2) as min_fare, round(max(Fare),2) as max_fare, round(avg(Fare),2) as mean_fare FROM table_df WHERE Age >= 18 GROUP BY Pclass'
spark.sql(sql_query).show()

+------+--------+--------+---------+
Pclass|min_fare|max_fare|mean_fare|
+------+--------+--------+---------+
 1| 0.0| 512.33| 90.9|
 3| 0.0| 56.5| 10.91|
 2| 9.69| 73.5| 20.72|
+------+--------+--------+---------+

+------+--------+--------+---------+
Pclass|min_fare|max_fare|mean_fare|
+------+--------+--------+---------+
 1| 0.0| 512.33| 90.9|
 3| 0.0| 56.5| 10.91|
 2| 9.69| 73.5| 20.72|
+------+--------+--------+---------+

### 2) Produce a summary with the survival rates for each passenger gender, with survival rate as a %?

In [10]:
# Aggregate by by gender and survived indicator
df_survival = df.groupBy(['Sex','Survived']).agg(f.count('Sex').alias('count')) \
                .withColumn('gender_total',f.sum(f.col('count')).over(Window.partitionBy(f.col('Sex'))))
df_survival.show()

# Calculate the survival rate
df_survival_rate = df_survival.withColumn('survival_rate (%)',f.round((f.col('count')/f.col('gender_total'))*100,2))\
                              .where(f.col('Survived') == 1)\
                              .select('Sex','survival_rate (%)')
df_survival_rate.show()

+------+--------+-----+------------+
 Sex|Survived|count|gender_total|
+------+--------+-----+------------+
female| 1| 385| 466|
female| 0| 81| 466|
 male| 0| 734| 843|
 male| 1| 109| 843|
+------+--------+-----+------------+

+------+-----------------+
 Sex|survival_rate (%)|
+------+-----------------+
female| 82.62|
 male| 12.93|
+------+-----------------+

### BONUS - Produce a summary of the passenger cabins. How will you handle missing values?

In [12]:
df_cabin = df.select('PassengerId','Pclass','Fare',
                    f.when(f.col('Cabin').isNull(),'N/A').otherwise(f.col('Cabin').substr(0,1)).alias('Cabin_Letter')
                    )
df_cabin.show(n=5)

+-----------+------+-------+------------+
PassengerId|Pclass| Fare|Cabin_Letter|
+-----------+------+-------+------------+
 1| 3| 7.25| N/A|
 2| 1|71.2833| C|
 3| 3| 7.925| N/A|
 4| 1| 53.1| C|
 5| 3| 8.05| N/A|
+-----------+------+-------+------------+
only showing top 5 rows

In [13]:
# Create a pivot table to show the classes who stay in each floor
df_cabin_summary = df_cabin.groupBy(['Cabin_Letter']).pivot('Pclass').agg(f.count("PassengerId")).orderBy('Cabin_Letter')

# Create 
df_cabin_fare = df_cabin.groupBy('Cabin_Letter').agg(f.round(f.mean('Fare'),2).alias('Avg_Price'))
df_cabin_summary = df_cabin_summary.join(df_cabin_fare,df_cabin_summary.Cabin_Letter == df_cabin_fare.Cabin_Letter,'left_outer').select(df_cabin_summary.Cabin_Letter,'1','2','3','Avg_Price')
df_cabin_summary.show()


+------------+----+----+----+---------+
Cabin_Letter| 1| 2| 3|Avg_Price|
+------------+----+----+----+---------+
 A| 22|null|null| 41.24|
 B| 65|null|null| 122.38|
 C| 94|null|null| 107.93|
 D| 40| 6|null| 53.01|
 E| 34| 4| 3| 54.56|
 F|null| 13| 8| 18.08|
 G|null|null| 5| 14.21|
 N/A| 67| 254| 693| 19.13|
 T| 1|null|null| 35.5|
+------------+----+----+----+---------+